# 背景建模

## 帧差法
由于场景中的目标在运动，目标的影响在不同的图像帧中的位置不同。该类算法对时间上连续的两帧图像进行查分运算，不同帧对应的像素点相减，判断灰度差的绝对值，当绝对值超过一定的阈值时，即可判断为运动的目标，从而实现目标的检测功能

## 混合高斯模型
在测试阶段，对新来的像素点值与混合高斯模型中的每一个均值进行比较，如果其差值在2倍的方差之间的话，则认为是背景，否则认为是前景。将前景赋值为255，背景赋值为0，这样就形成了一幅前景二值图

### 混合高斯模型学习方法
1、首先初始化每个高斯模型矩阵参数
2、取视频中T帧数据图像用来训练高斯混合模型，来了第一个像素之后用它来当做第一高斯分布
3、当后帧的像素值传来时，与前面已有的高斯的均值比较，如果该像素点的值与其模型均值差在3倍的方差内，则属于该分部，并对其进行参数更新
4、如果下一次来的像素不满足当前的高斯分布，用它来创建一个新的高斯分布

In [3]:
import numpy as np
import cv2

test_path = r'D:\\py_project\\test.avi'

In [5]:
#测试视频
cap = cv2.VideoCapture(test_path)
#形态学操作需要使用
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE,(3,3))
#创建混合高斯模型用于背景建模
fgbg = cv2.createBackgroundSubtractorMOG2()

while(True):
    ret, frame = cap.read()
    fgmask = fgbg.apply(frame)
    #形态学开运算去噪点
    fgmask = cv2.morphologyEx(fgmask, cv2.MORPH_OPEN, kernel)
    #寻找视频中的轮廓
    im, contours, hierarchy = cv2.findContours(fgmask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    for c in contours:
        #计算各轮廓的周长
        perimeter = cv2.arcLength(c,True)
        if perimeter > 188:
            #找到一个直矩形（不会旋转）
            x,y,w,h = cv2.boundingRect(c)
            #画出矩形
            cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)  

    cv2.imshow('frame', frame)
    cv2.imshow('fgmask', fgmask)
    k = cv2.waitKey(150) & 0xff
    if k == 27:
        break
            
cap.release()
cv2.destroyAllWindows()

# 光流估计
光流估计是空间运动物体在观测成像平面上的像素运动的瞬时速度，根据各个像素点的速度矢量特征，可以对图像进行动态分析，例如目标跟踪

*亮度恒定：同一点随着时间的变化，其亮度不会发生改变  

*小运动：随着时间的变化不会引起位置的剧烈变化，只有小运动的情况下才能用前后帧之间单位位置变化引起的灰度变化去近似灰度对位置的偏导数  

*空间一致：一个场景上临近的点投影到图像上也是离近点，且临近点速度一致。因为光流法基本方程约束只有一个，而要求x,y方向的速度，有两个位置变量。所以需要连立多个方程求解

In [7]:
cap = cv2.VideoCapture(test_path)

#角点检测所需参数
feature_params = dict(maxCorners = 100,qualityLevel = 0.3,minDistance = 7)

#lucas_kanade参数
lk_params = dict(winSize = (15,15),maxLevel = 2)

#随机颜色线条
color = np.random.randint(0,255,(100,3))

#拿到第一帧图像
ret, old_frame = cap.read()
old_gray = cv2.cvtColor(old_frame,cv2.COLOR_BGR2GRAY)
#返回所有检测特征点，需要输入图像，角点最大数量（效率），品质因子（特征值越大越好，来筛选）
#距离相当于这区间有比这个角点强的，就不要这个角点了
p0 = cv2.goodFeaturesToTrack(old_gray, mask = None, **feature_params)

#创建一个mask
mask = np.zeros_like(old_frame)

while(True):
    ret,frame = cap.read()
    frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #需要传入前一帧和当前图像以及前一帧检测到的角点
    pl,st,err = cv2.calcOpticalFlowPyrLK(old_gray,frame_gray,p0,None, **lk_params)
    
    #st=1表示
    good_new = pl[st==1]
    good_old = p0[st==1]
    
    #绘制轨迹
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv2.line(mask,(a,b),(c,d),color[i].tolist(),2)
        frame = cv2.circle(frame,(a,b),5,color[i].tolist(),-1)
    img = cv2.add(frame,mask)
    
    cv2.imshow('frame',img)
    k = cv2.waitKey(150) & 0xff
    if k == 27:
        break
    
    #更新
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    
cv2.destroyAllWindows()
cap.release()